<a href="https://colab.research.google.com/github/junnu01/Types-of-Optimizers-in-AI/blob/main/OptimizationISE_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import time
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# === Load Dataset ===
data = load_breast_cancer()
X = data.data
y = data.target

# Normalizing the dataset
X = (X - X.min(0)) / (X.max(0) - X.min(0))

# === Objective Function for Feature Selection ===
def fitness_function(position, X_train, X_test, y_train, y_test):
    selected_features = [i for i in range(len(position)) if position[i] > 0.5]
    if len(selected_features) == 0:
        return 1  # Penalize empty selection
    clf = KNeighborsClassifier()
    clf.fit(X_train[:, selected_features], y_train)
    y_pred = clf.predict(X_test[:, selected_features])
    accuracy = accuracy_score(y_test, y_pred)
    return 1 - accuracy  # Since we minimize in optimizers

# === Optimizer Template ===
def run_optimizer(optimizer_func, X, y, name):
    runs = 5
    acc_list = []
    times = []
    for _ in range(runs):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
        start_time = time.time()
        best_position, best_score = optimizer_func(lambda pos: fitness_function(pos, X_train, X_test, y_train, y_test), X.shape[1])
        duration = time.time() - start_time
        acc = 1 - best_score
        acc_list.append(acc)
        times.append(duration)
    print(f"\n{name} Optimizer")
    print("Average Accuracy:", np.mean(acc_list))
    print("Average Time (s):", np.mean(times))
    print("Standard Deviation:", np.std(acc_list))

# === Snow Ablation Optimizer ===
def SAO(fitness, dim, n=30, max_iter=50):
    lb, ub = 0, 1
    X = np.random.rand(n, dim)
    fit = np.array([fitness(x) for x in X])
    best = X[np.argmin(fit)]
    for t in range(max_iter):
        for i in range(n):
            alpha = np.random.rand()
            beta = np.random.rand()
            X[i] = X[i] + alpha * (best - abs(X[i])) + beta * (np.random.rand(dim) * (ub - lb) + lb)
            X[i] = np.clip(X[i], lb, ub)
            f = fitness(X[i])
            if f < fitness(best):
                best = X[i]
    return best, fitness(best)

# === Spider Wasp Optimizer ===
def SWO(fitness, dim, n=30, max_iter=50):
    lb, ub = 0, 1
    X = np.random.rand(n, dim)
    fit = np.array([fitness(x) for x in X])
    best = X[np.argmin(fit)]
    for t in range(max_iter):
        for i in range(n):
            rand1 = np.random.rand(dim)
            rand2 = np.random.rand(dim)
            X[i] = X[i] + rand1 * (best - X[i]) + rand2 * (np.random.rand(dim))
            X[i] = np.clip(X[i], lb, ub)
            f = fitness(X[i])
            if f < fitness(best):
                best = X[i]
    return best, fitness(best)

# === Energy Valley Optimizer ===
def EVO(fitness, dim, n=30, max_iter=50):
    lb, ub = 0, 1
    X = np.random.rand(n, dim)
    fit = np.array([fitness(x) for x in X])
    best = X[np.argmin(fit)]
    for t in range(max_iter):
        for i in range(n):
            r = np.random.rand()
            valley = r * (best - X[i]) + (1 - r) * (np.random.rand(dim))
            X[i] = X[i] + valley
            X[i] = np.clip(X[i], lb, ub)
            f = fitness(X[i])
            if f < fitness(best):
                best = X[i]
    return best, fitness(best)

# === Run All Optimizers ===
run_optimizer(SAO, X, y, "Snow Ablation")
run_optimizer(SWO, X, y, "Spider Wasp")
run_optimizer(EVO, X, y, "Energy Valley")



Snow Ablation Optimizer
Average Accuracy: 0.9695906432748538
Average Time (s): 37.31910285949707
Standard Deviation: 0.010061199142739898

Spider Wasp Optimizer
Average Accuracy: 0.9660818713450293
Average Time (s): 34.06024351119995
Standard Deviation: 0.014514238182445453

Energy Valley Optimizer
Average Accuracy: 0.9649122807017545
Average Time (s): 33.71044297218323
Standard Deviation: 0.011095711088310103
